In [6]:
from foursquare_credentials import CLIENT_ID, CLIENT_SECRET, LIMIT, VERSION
import math, requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd

In [7]:
def get_circe_radius(area):
    '''Accepts area in square kilometers and gives radius in meters as an output'''
    return int(math.sqrt(area/math.pi) * 1000)

In [8]:
def get_category_type(row):
    '''function that extracts the category of the venue'''
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def get_nearby_venues(names, latitudes, longitudes, areas):
    
    venues_list=[]
    for name, lat, lng, area in zip(names, latitudes, longitudes, areas):
        print(name)
        radius = get_circe_radius(area) * 0.8
        # create the API request URL
        url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
berlin_regions = pd.read_csv('berlin_regions.csv', sep=';')
berlin_regions.head()

,Unnamed: 0,Postal Code,City,Administrative Region,Population,Area km²,Population density per km²,Median Age,Neighborhoods,Latitude,Longitude
0,0,10115,Berlin,Berlin,16678,2.421,6888.89,43.0,Mitte,52.533707,13.387224
1,1,10117,Berlin,Berlin,24223,3.321,7293.89,43.0,Mitte,52.518746,13.390193
2,2,10119,Berlin,Berlin,7408,0.857,8644.11,43.0,"Bezirk Pankow, Mitte",52.532666,13.407149
3,3,10178,Berlin,Berlin,14069,1.872,7515.49,43.0,Mitte,52.523474,13.412203
4,4,10179,Berlin,Berlin,15897,2.183,7282.18,43.0,"Luisenstadt, Mitte",52.514591,13.419699


In [ ]:
berlin_venues = get_nearby_venues(
    names=berlin_regions['Neighborhoods'],
    latitudes=berlin_regions['Latitude'],
    longitudes=berlin_regions['Longitude'],
    areas=berlin_regions['Area km²']
)